# Notes

code for producing GOIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [1]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [30]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

In [53]:
#fuel_type = 'Gas'
fuel_type = 'Oil'
#fuel_type = 'NGL'

## import data

In [54]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
if fuel_type=='Gas':
    pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)
if fuel_type=='Oil':
    pipes_df_orig = oil_pipes.copy()

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel==fuel_type]

owners_df_orig = owners_df_orig.loc[owners_df_orig['ProjectID']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']

owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [55]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/1702877721.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  country_ratios_df.replace('--', numpy.nan, inplace=True)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/1702877721.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  owners_df_orig.replace('',numpy.nan,inplace=True)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/1702877721.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a fut

In [56]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

#region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [57]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([('Africa',    'Northern Africa'),
            ('Africa', 'Sub-Saharan Africa')],
           names=['Region', 'SubRegion'])

## file names with regional specifics

In [58]:
if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-gas-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-NGL-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-Oil-pipelines-'+str(datetime.date.today())+'.xlsx')

### create country-specific dataframes for region, country_ratios_df, owners_df

In [59]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

In [60]:
pipes_df_touse.head()

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Fuel,Countries,Status,PipelineType,Researcher,...,RouteMapNotes,OtherEnglishNames,OtherLanguagePrimaryPipelineName,OtherLanguageAlternativePipelineNames,OtherLanguageSegmentName,OtherLanguageWikiPage,AlternateRouteProjectIDs,CharacterCountTooLong,CostUSDPerKm,CostEuroPerKm
3,Ezanga Oil Pipeline Network: https://www.gem.w...,Rabi-Cap Lopez Oil Pipeline,,https://www.gem.wiki/Rabi-Cap_Lopez_Oil_Pipeline,P3685,Oil,Gabon,Operating,,CJ,...,https://www.abid.co/sites/default/files/201901...,,,,,,,,NaN,NaN
4,Ezanga Oil Pipeline Network: https://www.gem.w...,Rabi-Gamba Oil Pipeline,,https://www.gem.wiki/Rabi-Gamba_Oil_Pipeline,P3686,Oil,Gabon,Operating,,CJ,...,https://www.abid.co/sites/default/files/201901...,,,,,,,,NaN,NaN
5,Ezanga Oil Pipeline Network: https://www.gem.w...,Onal-Coucal Oil Pipeline,,https://www.gem.wiki/Onal-Coucal_Oil_Pipeline,P3687,Oil,Gabon,Operating,,CJ,...,https://www.abid.co/sites/default/files/201901...,,,,,,,,NaN,NaN
15,,Lamu Port-South Sudan (LAPSSET) Pipeline,South Sudan–Lokichar Oil Pipeline Segment,https://www.gem.wiki/Lamu_Port-South_Sudan_(LA...,P0538,Oil,"South Sudan, Kenya",Proposed,,CJ,...,https://pp2.au-pida.org/approved-project/?entr...,Lokichar to Lamu Crude Oil Pipeline (LLCOP),,,,,,,NaN,NaN
37,,Durban–Sasolburg Oil Pipeline,,https://www.gem.wiki/Durban%E2%80%93Sasolburg_...,P2468,Oil,South Africa,Operating,,CJ,...,https://www.transnet.net/BusinessWithUs/LTPF%2...,,,,,,,,NaN,NaN


### sum MergedKmByCountry and MergedKmByRegion

In [61]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [62]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [63]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/3633674232.py:40: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_region.to_excel(excel_writer, 'Kilometers by region')
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/3633674232.py:41: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_country.to_excel(excel_writer, 'Kilometers by country')


In [64]:
km_by_region

Proposed  Construction  \
Region Subregion                                    
Africa Northern Africa        68.50         100.0   
       Sub-Saharan Africa   6285.75        1950.0   
Total                       6354.25        2050.0   

                           In Development (Proposed + Construction)  Shelved  \
Region Subregion                                                               
Africa Northern Africa                                       168.50   483.20   
       Sub-Saharan Africa                                   8235.75   502.92   
Total                                                       8404.25   986.12   

                           Cancelled  Operating Idle Mothballed Retired  
Region Subregion                                                         
Africa Northern Africa        620.04   15509.69                          
       Sub-Saharan Africa    2100.77    7457.52                          
Total                        2720.81   22967.21

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

In [65]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ProjectID)
    parent_string = row.Parent
    #print(parent_string)

    #print(parent_string)
    # if the first letter of the parent_string is a chinese character, and it ends with [100.00%],
    # that means it's a non-researched QCC owner; keep as-is
    if re.findall(r'[\u4e00-\u9fff]+', parent_string[:1]) != [] and parent_string[-9:]=='[100.00%]':
        parent_list = [parent_string.split(' [100.00%]')[0]]
        percent_list = [1.0]
    # otherwise do the rest of the thing
    else:
        parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
        percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        #print(parent_list)
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent,'ParentHQCountry'],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent,'ParentHQRegion'],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'MergedKmByCountry':row.MergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.MergedKmByCountry

In [66]:
owner_parent_calculations_df

,Parent,ProjectID,FractionOwnership,ParentHQCountry,ParentHQRegion,Country,Status,MergedKmByCountry,KmOwnership
0,Santos Limited,P1077,1.00,Australia,Oceania,Papua New Guinea,Operating,265.0,265.0
0,Sonatrach,P0524,1.00,Algeria,Africa,Algeria,Operating,821.0,821.0
0,Sonatrach,P0525,1.00,Algeria,Africa,Algeria,Operating,646.0,646.0
0,Egyptian General Petroleum Corporation,P0530,0.50,Egypt,Africa,Egypt,Operating,320.0,160.0
0,Mubadala Investment Company,P0530,0.15,United Arab Emirates,Asia,Egypt,Operating,320.0,48.0
...,...,...,...,...,...,...,...,...,...
0,Perenco,P3685,1.00,United Kingdom,Europe,Gabon,Operating,235.0,235.0
0,Carlyle Group,P3686,1.00,United States,Americas,Gabon,Operating,120.0,120.0
0,Maurel & Prom,P3687,1.00,France,Europe,Gabon,Operating,120.0,120.0
0,unknown,P5239,1.00,unknown,--,Angola,Proposed,700.0,700.0


In [67]:
#unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = \
    owner_parent_calculations_df.groupby(
    ['Parent','ParentHQCountry','Status'])[['KmOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list)
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_km_by_status_df = owners_km_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_km_by_status_df.loc['Total',:] = owners_km_by_status_df.sum(axis=0, min_count=0).values
owners_km_by_status_df.loc['Total','Country'] = ''

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Kilometers by owner')

In [68]:
owners_km_by_status_df

,Country,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent,,,,,,,,,,
APA Corporation,United States,,,,,,176.838,,,
Africa Oil,Canada,432.9375,,432.9375,,,,,,
Aiteo Group,Nigeria,,,,,,43.65,,,
CNOOC Limited,China,115.52,,115.52,,500.0,,,,
Carlyle Group,United States,,,,,,120.0,,,
Chevron,United States,,,,,,359.0,,,
China National Petroleum Corporation,China,,1950.0,1950.0,,300.385,639.156,,,
China Petrochemical Corporation,China,,,,,,46.662,,,
ConocoPhillips,United States,,,,,,155.931904,,,


### pipeline km by start year, type

In [69]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Gas')]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Oil')]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='NGL')]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [70]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Oil pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_87773/4193317879.py:21: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')


## save excel file

In [71]:
excel_writer.close()

## calculating stats for landing page

In [72]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type]['LengthMergedKm'].sum()/1e6, 'M km tracked')

107 gas pipeline projects tracked
0.03534810000000001 M km tracked


In [73]:
69400/161

431.055900621118

In [74]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

8 gas pipeline projects tracked
8.40724 K km tracked
